python version: 3.11.6

# Misc Startup Codes

In [3]:
import pandas as pd
import warnings


In [4]:
cleanETFtable = pd.read_csv("./CleanETFTableUpdated.csv")
fundValue = pd.read_csv("./JPMValue.csv")
cleanETFtable = cleanETFtable.drop(['Unnamed: 0'],axis=1)
fundValue = fundValue.drop(['Unnamed: 0'],axis=1)

# Ignore warnings
warnings.filterwarnings('ignore')


# Convert the dataframe to a multi-index dataframe for easier and simple accessing

In [11]:
filingPeriodList = cleanETFtable['Period of Report'].unique() #filing periods
# filingPeriod
lastTwoFiling = filingPeriodList[:2]
cleanETFtable = cleanETFtable[cleanETFtable['Period of Report'].isin(lastTwoFiling)]# comment this if u want all filings 


In [12]:
CompanyList= fundValue['Company Name'].to_list()
CompanyReviewList = set(cleanETFtable[cleanETFtable['Period of Report']=='2023-06-30']['Company Name']).intersection(set(cleanETFtable[cleanETFtable['Period of Report']!='2023-06-30']['Company Name']))

#CompanyList

In [13]:
cleanETFtableCompare = cleanETFtable[cleanETFtable['Company Name'].isin(list(CompanyReviewList))]


In [14]:
cleanETFtableCompare.keys()

Index(['VALUE', 'Company Name', 'Period of Report', 'Amend',
       'ETF Security Description', 'ETF Agent', 'Foreign (F) / Domestic (D)',
       'ETF Agent Name', 'JPM Product', 'JPM Brokerage', 'Portfolio ID',
       'TOTAL VALUE', 'Holding Weight', 'Trading Symbol', 'Issuer', 'Theme',
       'Strategy', 'Geography', 'Asset Class', 'Sector/Focus'],
      dtype='object')

Feel free to change the indexing way

In [327]:
multiClient = cleanETFtableCompare.set_index(['Company Name',
                                              'Period of Report',
                                            'JPM Brokerage',
                                              'Issuer',
                                              'Asset Class',
                                              'Strategy',
                                              'Foreign (F) / Domestic (D)',
],drop=False)

In [278]:
#product level query
multiCompetitor = cleanETFtableCompare.set_index([
                                              'Issuer',
                                              'Period of Report',
                                              'ETF Security Description',
                                              'Company Name',
                                              'Asset Class'
                                              ],drop=False)
#competitor-client query
multiCompetitor2 = cleanETFtableCompare.set_index([
                                              'Issuer',
                                              'Period of Report',
                                              'Company Name',
                                              'Asset Class',
                                              'Strategy',
                                              'Sector/Focus',
                                              'Geography'
                                              ],drop=False)

# Compare Holding Changes across q1 and q2

## Understanding Client's Flow

### e.g. Compare total portfolio value change 

In [328]:
client  = 'IMS Capital Management'
df = multiClient.loc[client] #assuming this is one of our top client

print(f"Previous Holding: {df.loc['2023-03-31']['TOTAL VALUE'][0]}")
print(f"New Holding: {df.loc['2023-06-30']['TOTAL VALUE'][0]}")


Previous Holding: 22514432
New Holding: 24641526


### e.g. Compare JPM product change (net flow, net weight, num of pos)

In [348]:
af = df.loc['2023-06-30'][df.loc['2023-06-30']['JPM Product']==True]
af_market = df.loc['2023-06-30'].groupby(level=['Strategy'])['VALUE'].sum() #change the parameter in level to other market factors 
af_market_pct = df.loc['2023-06-30'].groupby(level=['Strategy'])['Holding Weight'].sum() #change the parameter in level to other market factors 

In [349]:
b4 = df.loc['2023-03-31'][df.loc['2023-03-31']['JPM Product']==True]
b4_market = df.loc['2023-03-31'].groupby(level=['Strategy'])['VALUE'].sum() #change the parameter in level to other market factors 
b4_market_pct = df.loc['2023-03-31'].groupby(level=['Strategy'])['Holding Weight'].sum() #change the parameter in level to other market factors 

In [352]:
print(f"Net JPM product inflow(outflow): ${af['VALUE'].sum()-b4['VALUE'].sum()} USD")
print(f"Net JPM product weight increase(decrease): {(af['Holding Weight'].sum()-b4['Holding Weight'].sum())*100} %")
print(f"New JPM holding(s): {set(af['ETF Security Description']).difference(set(b4['ETF Security Description']))}")
print(f"Unloaded JPM holding(s): {set(b4['ETF Security Description']).difference(set(af['ETF Security Description']))}")

Net JPM product inflow(outflow): $205214 USD
Net JPM product weight increase(decrease): -0.8720620496075498 %
New JPM holding(s): {'J P MORGAN EXCHANGE-TRADED FD TR EQU'}
Unloaded JPM holding(s): set()


In [347]:
af_market-b4_market # shows the reported fund inflow (outflow) across markets across the two quarters

Strategy
Active               205214
Dividends            231739
ESG                   41327
Exchange-specific     43036
Growth               885046
Multi-factor          -3596
Value                -19830
Vanilla              744158
Name: VALUE, dtype: int64

In [351]:
af_market_pct - b4_market_pct *100 # shows the reported portfolio weight increase (decrease) across markets across the two quarters

Strategy
Active              -19.561329
Dividends           -20.475496
ESG                  -3.339718
Exchange-specific    -1.100157
Growth              -39.365434
Multi-factor         -3.687504
Value                -1.099874
Vanilla             -10.370487
Name: Holding Weight, dtype: float64

### e.g. Compare non-JPM product change (net flow, net weight, num of pos)

In [173]:
af = df.loc['2023-06-30'][df.loc['2023-06-30']['JPM Product']==False]


In [338]:
af

,,,VALUE,Company Name,Period of Report,Amend,ETF Security Description,ETF Agent,Foreign (F) / Domestic (D),ETF Agent Name,JPM Product,JPM Brokerage,Portfolio ID,TOTAL VALUE,Holding Weight,Trading Symbol,Issuer,Theme,Strategy,Geography,Asset Class,Sector/Focus
ETF Security Description,Company Name,Asset Class,,,,,,,,,,,,,,,,,,,,
ISHARES INC CORE MSCI EMERGING MKTS,"DecisionPoint Financial, LLC",Equity,8183,"DecisionPoint Financial, LLC",2023-06-30,False,ISHARES INC CORE MSCI EMERGING MKTS,2767.0,F,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"DecisionPoint Financial, LLC 2023-06-30",157897624,0.000052,IEMG,Blackrock Financial Management,Equity: Emerging Markets - Total Market,Vanilla,Broad Asia,Equity,Total Market
ISHARES 0-5 YEAR INVESTMENT GRADE CO,"DecisionPoint Financial, LLC",Bond,4988241,"DecisionPoint Financial, LLC",2023-06-30,False,ISHARES 0-5 YEAR INVESTMENT GRADE CO,2767.0,D,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"DecisionPoint Financial, LLC 2023-06-30",157897624,0.031592,SLQD,Blackrock Financial Management,"Fixed Income: U.S. - Corporate, Broad-based In...",Vanilla,North America,Bond,Investment Grade
ISHARES TR CORE MSCI EAFE ETF,"DecisionPoint Financial, LLC",Equity,5130,"DecisionPoint Financial, LLC",2023-06-30,False,ISHARES TR CORE MSCI EAFE ETF,2767.0,F,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"DecisionPoint Financial, LLC 2023-06-30",157897624,0.000032,IEFA,Blackrock Financial Management,Equity: Developed Markets Ex-North America - ...,Vanilla,Developed Markets,Equity,Total Market
ISHARES CORE S&P MID-CAP ETF,"DecisionPoint Financial, LLC",Equity,21965,"DecisionPoint Financial, LLC",2023-06-30,False,ISHARES CORE S&P MID-CAP ETF,2869.0,D,BNYMELLON/RE ETF - ISHARES DTC/NSCC,False,False,"DecisionPoint Financial, LLC 2023-06-30",157897624,0.000139,IJH,Blackrock Financial Management,Equity: U.S. - Mid Cap,Vanilla,North America,Equity,Mid Cap
ISHARES CORE S&P SM CAP ETF,"DecisionPoint Financial, LLC",Equity,5681,"DecisionPoint Financial, LLC",2023-06-30,False,ISHARES CORE S&P SM CAP ETF,2869.0,D,BNYMELLON/RE ETF - ISHARES DTC/NSCC,False,False,"DecisionPoint Financial, LLC 2023-06-30",157897624,0.000036,IJR,Blackrock Financial Management,Equity: U.S. - Small Cap,Vanilla,North America,Equity,Small Cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISHARES BARCLAYS 1-3 YEAR TREASURY B,"Artemis Wealth Advisors, LLC",Bond,7784,"Artemis Wealth Advisors, LLC",2023-06-30,False,ISHARES BARCLAYS 1-3 YEAR TREASURY B,2767.0,D,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"Artemis Wealth Advisors, LLC 2023-06-30",853388176,0.000009,SHY,Blackrock Financial Management,"Fixed Income: U.S. - Government, Treasury Inve...",Vanilla,North America,Bond,Investment Grade
ISHARES TRUST ISHARES 1-5 YEAR INVES,"WASHINGTON CAPITAL MANAGEMENT, INC",Bond,2730151,"WASHINGTON CAPITAL MANAGEMENT, INC",2023-06-30,False,ISHARES TRUST ISHARES 1-5 YEAR INVES,2767.0,D,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"WASHINGTON CAPITAL MANAGEMENT, INC 2023-06-30",102012320,0.026763,IGSB,Blackrock Financial Management,"Fixed Income: U.S. - Corporate, Broad-based In...",Vanilla,Developed Markets,Bond,Investment Grade
ISHARES MSCI ACWI EX US ETF,"WASHINGTON CAPITAL MANAGEMENT, INC",Equity,1863469,"WASHINGTON CAPITAL MANAGEMENT, INC",2023-06-30,False,ISHARES MSCI ACWI EX US ETF,2767.0,F,SSB - BLACKROCK INSTITUTIONAL TRUST,False,False,"WASHINGTON CAPITAL MANAGEMENT, INC 2023-06-30",102012320,0.018267,ACWX,Blackrock Financial Management,Equity: Global Ex-U.S. - Total Market,Vanilla,Broad Asia,Equity,Total Market


In [172]:
b4 = df.loc['2023-03-31'][df.loc['2023-03-31']['JPM Product']==False]

In [56]:
print(f"Net non-JPM product inflow(outflow): ${af['VALUE'].sum()-b4['VALUE'].sum()} USD")
print(f"Net non-JPM product weight increase(decrease): {(af['Holding Weight'].sum()-b4['Holding Weight'].sum())*100} bps")
print(f"New non-JPM holding(s): {set(af['ETF Security Description']).difference(set(b4['ETF Security Description']))}")
print(f"Unloaded non-JPM holding(s): {set(b4['ETF Security Description']).difference(set(af['ETF Security Description']))}")

Net non-JPM product inflow(outflow): $1921880 USD
Net non-JPM product weight increase(decrease): 0.8720620496075138 bps
New non-JPM holding(s): {'VANGUARD TOTAL STOCK MARKET ETF'}
Unloaded non-JPM holding(s): set()


### e.g. Identify non-JPM positions traded thru JPM broker (any open position), possibly pitch them similar products and advantage is lower transaction fee

In [329]:
#traded thru JPM
df.loc['2023-06-30',True][df.loc['2023-06-30',True]['JPM Product']==False]['ETF Security Description']

Issuer                          Asset Class  Strategy  Foreign (F) / Domestic (D)
Blackrock Financial Management  Equity       Vanilla   D                                    ISHARES RUSSELL MID-CAP ETF
                                             Value     D                               ISHARES S&P MIDCAP 400 VALUE ETF
Vanguard                        Equity       Vanilla   D                                VANGUARD TOTAL STOCK MARKET ETF
Blackrock Financial Management  Equity       Growth    D                             ISHARES RUSSELL MID-CAP GROWTH ETF
Name: ETF Security Description, dtype: object

In [330]:
#not traded thru JPM
df.loc['2023-06-30',False][df.loc['2023-06-30',False]['JPM Product']==False]['ETF Security Description']

Issuer                          Asset Class  Strategy           Foreign (F) / Domestic (D)
Invesco                         Equity       Multi-factor       D                              INVESCO ETF TR II S&P 500 HIGH DIVD
                                             Exchange-specific  D                                        INVESCO QQQ TR UNIT SER 1
Blackrock Financial Management  Equity       Vanilla            D                                     ISHARES CORE S&P MID-CAP ETF
                                                                D                                      ISHARES CORE S&P SM CAP ETF
                                             Growth             D                                      ISHARES RUSSELL 1000 GR ETF
Wisdomtree                      Equity       Dividends          D                             WISDOMTREE TR U.S. MIDCAP DIVID FD (
State Street                    Equity       Vanilla            D                                       SPDR S&P 500 ETF TR

## Understanding Client's Holding

In [373]:
df.loc['2023-06-30']['Holding Weight'].groupby('Issuer').sum()

KeyError: 'Issuer'

In [353]:
#client's top 10 holding, 
cleanETFtableCompare.set_index(['Company Name',
                                              'Period of Report',
                                              'ETF Security Description'
                                              ]).loc[client].loc['2023-06-30']['VALUE'].sort_values(ascending=False).head(n=10)

ETF Security Description
ISHARES RUSSELL 1000 GR ETF             7201096
JP MORGAN ETF TR ULTRA-SHORT INCOME     3085981
ISHARES RUSSELL MID-CAP GROWTH ETF      2636740
ISHARES TR SELECT DIVID ETF             2109646
VANGUARD DIVIDEND APPRECIATION ETF      1537887
WISDOMTREE TR U.S. MIDCAP DIVID FD (    1238793
ISHARES CORE S&P SM CAP ETF              863077
INVESCO ETF TR II S&P 500 HIGH DIVD      834247
J P MORGAN EXCHANGE-TRADED FD TR COR     809560
SPDR S&P 500 ETF TR UNIT S               672608
Name: VALUE, dtype: int64

In [371]:
#client's top 10 holding by market, Change the last index "Asset Class" to others such as "Strategy" for other types of indexing, also change the parameter after loc to select the subcategory 
cleanETFtableCompare.set_index(['Company Name',
                                              'Period of Report',
                                              'Asset Class'
                                              ]).loc[client].loc['2023-06-30'].loc['Bond']["ETF Security Description"]

Asset Class
Bond    ISHARES IBOXX $ INVESTMENT GRADE COR
Bond    ISHARES ESG AWARE U.S. AGGREGATE BON
Bond    J P MORGAN EXCHANGE-TRADED FD TR COR
Bond     JP MORGAN ETF TR ULTRA-SHORT INCOME
Bond    JP MORGAN ULTRA SHORT MUN INCOME ETF
Name: ETF Security Description, dtype: object

## Competitior Analysis 

In [363]:
#e.g. finding out major inflows / outflows of Blackrock 
dff = multiCompetitor.loc['Blackrock Financial Management']
af= dff.loc['2023-06-30']
b4 = dff.loc['2023-03-31']

In [364]:
#identifying major fund inflow (in monetary value) of Blackrock's product
(af['VALUE'].groupby(level=0).sum()-b4['VALUE'].groupby(level=0).sum()).sort_values(ascending=False ).head(n=10)

ETF Security Description
ISHARES RUSSELL 1000 GR ETF             827844871.0
ISHARES IBOXX $ INVESTMENT GRADE COR    743784744.0
ISHARES CORE S&P 500 ETF                646895251.0
ISHARES MSCI USA MIN VOL FACTOR ETF     407099528.0
ISHARES BARCLAYS 7-10 YEAR TREASURY     364474886.0
ISHARES MSCI USA QUALITY FACTOR ETF     312885263.0
ISHARES BARCLAYS 1-3 YEAR TREASURY B    217394320.0
ISHARES TR RUSSELL 1000 ETF             213311011.0
ISHARES TR EXPONENTIAL TECHNOLOGIES     191508051.0
ISHARES CORE U.S. AGGREGATE BOND ETF    180511873.0
Name: VALUE, dtype: float64

In [365]:
#identifying major fund outflow (in monetary value) of Blackrock's product

(b4['VALUE'].groupby(level=0).sum()-af['VALUE'].groupby(level=0).sum()).sort_values(ascending=False ).head(n=10)

ETF Security Description
ISHARES TRUST ISHARES 1-5 YEAR INVES    267412630.0
ISHARES TR IBOXX $ HIGH YIELD CORP      219095459.0
ISHARES MSCI USA MOMENTUM FACTOR ETF    206552707.0
ISHS MSCI UTD KINGDOM ETF               133552723.0
ISHARES TR BROAD USD HIGH YIELD CORP     39210699.0
ISHARES MSCI ALL COUNTRY ASIA EX JAP     35000744.0
ISHARES EXPANDED TECH-SOFTWARE SECTO     19201789.0
ISHARES S&P SMALL CAP 600 GROWTH ETF     17906831.0
ISHARES TR TIPS BD ETF                   16809874.0
ISHARES S&P SMALL-CAP 600 VALUE ETF      15053878.0
Name: VALUE, dtype: float64

In [366]:
#identifying major holders (in % of issuer's weight in total portfolio) of Blackrock 
af['Holding Weight'].groupby(level=0).sum().sort_values(ascending=False ).head(n=10)

ETF Security Description
ISHARES MSCI USA QUALITY FACTOR ETF     1.454932
ISHARES CORE S&P 500 ETF                1.451354
ISHARES MSCI CANADA ETF                 1.186808
ISHARES RUSSELL 1000 GR ETF             1.099338
ISHARES TR MSCI EAFE ETF                0.839991
ISHARES CORE U.S. AGGREGATE BOND ETF    0.779096
ISHARES CORE S&P MID-CAP ETF            0.772546
ISHARES INC MSCI SOUTH KOREA ETF        0.708414
ISHARES RUSSELL 2000 ETF                0.700243
ISHARES CORE S&P TOTAL U.S. STK MARK    0.679663
Name: Holding Weight, dtype: float64

In [367]:
#top clients of this product
af.loc['ISHARES INC CORE MSCI EMERGING MKTS']['VALUE'].groupby(level=0).sum().sort_values(ascending=False ).head(n=10)

Company Name
FMR LLC                                   791363048
UNITED CAPITAL FINANCIAL ADVISERS, LLC    232042209
Orion Portfolio Solutions, LLC             79380828
Atria Wealth Solutions, Inc.               24946729
Halpern Financial, Inc.                     9551070
Scarborough Advisors, LLC                   9190652
Delap Wealth Advisory, LLC                  4651300
Farther Finance Advisors, LLC               4456725
SCS Capital Management LLC                  3441946
Western Wealth Management, LLC              2660885
Name: VALUE, dtype: int64

In [368]:
#identifying major holders (in monetary value of issuer's weight in total portfolio) of Blackrock 
df_2 = multiCompetitor2.loc['Blackrock Financial Management']
af_2= df_2.loc['2023-06-30']
b4_2 = df_2.loc['2023-03-31']
af_2 ['VALUE'].groupby(level=0).sum().sort_values(ascending=False ).head(n=10)

Company Name
FMR LLC                                   40412679917
UNITED CAPITAL FINANCIAL ADVISERS, LLC     4424068588
Orion Portfolio Solutions, LLC             3223734104
Atria Wealth Solutions, Inc.               1429601752
SCS Capital Management LLC                 1097096654
VANGUARD GROUP INC                          548792146
Winthrop Advisory Group LLC                 262788202
OVERSEA-CHINESE BANKING Corp Ltd            214130295
Halpern Financial, Inc.                     192729928
BW Gestao de Investimentos Ltda.            169574936
Name: VALUE, dtype: int64

In [369]:
#identifying major holders (in % of issuer's weight in total portfolio) of Blackrock 
af_2['Holding Weight'].groupby(level=0).sum().sort_values(ascending=False ).head(n=10)

Company Name
VIRGINIA RETIREMENT SYSTEMS ET AL    1.000000
CLOVERFIELDS CAPITAL GROUP, LP       1.000000
W ADVISORS, LLC                      0.846410
FMR LLC                              0.772568
Halpern Financial, Inc.              0.702592
STRS OHIO                            0.684047
CAPE ANN SAVINGS BANK                0.677760
Mondrian Investment Partners LTD     0.654539
SlateStone Wealth, LLC               0.608812
IMS Capital Management               0.606974
Name: Holding Weight, dtype: float64

In [372]:
#type of products being held (change 'Asset Class' for other indexing parameters)
af_2

VALUE  \
Company Name                       Asset Class Strategy Sector/Focus     Geography                    
DecisionPoint Financial, LLC       Equity      Vanilla  Total Market     Broad Asia            8183   
                                   Bond        Vanilla  Investment Grade North America      4988241   
                                   Equity      Vanilla  Total Market     Developed Markets     5130   
                                                        Mid Cap          North America        21965   
                                                        Small Cap        North America         5681   
...                                                                                             ...   
Artemis Wealth Advisors, LLC       Bond        Vanilla  Investment Grade North America         7784   
WASHINGTON CAPITAL MANAGEMENT, INC Bond        Vanilla  Investment Grade Developed Markets  2730151   
                                   Equity      Vanilla  Total Market     Broad Asia         1863469   
                                                        Small Cap        North America       365177   
ARMSTRONG HENRY H ASSOCIATES INC   Equity      Vanilla  Large Cap        North America       233998   

                                                                                                                  Company Name  \
Company Name                       Asset Class Strategy Sector/Focus     Geography                                               
DecisionPoint Financial, LLC       Equity      Vanilla  Total Market     Broad Asia               DecisionPoint Financial, LLC   
                                   Bond        Vanilla  Investment Grade North America            DecisionPoint Financial, LLC   
                                   Equity      Vanilla  Total Market     Developed Markets        DecisionPoint Financial, LLC   
                                                        Mid Cap          North America            DecisionPoint Financial, LLC   
                                                        Small Cap        North America            DecisionPoint Financial, LLC   
...                                                                                                                        ...   
Artemis Wealth Advisors, LLC       Bond        Vanilla  Investment Grade North America            Artemis Wealth Advisors, LLC   
WASHINGTON CAPITAL MANAGEMENT, INC Bond        Vanilla  Investment Grade Developed Markets  WASHINGTON CAPITAL MANAGEMENT, INC   
                                   Equity      Vanilla  Total Market     Broad Asia         WASHINGTON CAPITAL MANAGEMENT, INC   
                                                        Small Cap        North America      WASHINGTON CAPITAL MANAGEMENT, INC   
ARMSTRONG HENRY H ASSOCIATES INC   Equity      Vanilla  Large Cap        North America        ARMSTRONG HENRY H ASSOCIATES INC   

                                                                                           Period of Report  \
Company Name                       Asset Class Strategy Sector/Focus     Geography                            
DecisionPoint Financial, LLC       Equity      Vanilla  Total Market     Broad Asia              2023-06-30   
                                   Bond        Vanilla  Investment Grade North America           2023-06-30   
                                   Equity      Vanilla  Total Market     Developed Markets       2023-06-30   
                                                        Mid Cap          North America           2023-06-30   
                                                        Small Cap        North America           2023-06-30   
...                                                                                                     ...   
Artemis Wealth Advisors, LLC       Bond        Vanilla  Investment Grade North America           2023-06-30   
WASHINGTON CAPITAL MANAGEMENT, INC Bond        Vanilla  Investme

## Landscape

In [411]:
df_3 = cleanETFtableCompare.set_index([
                                              'Period of Report',
                                              ],drop=False)

net_chg = df_3.loc['2023-06-30'].groupby('Issuer')['VALUE'].sum() - df_3.loc['2023-03-31'].groupby('Issuer')['VALUE'].sum() 

In [395]:
#top fund inflow by issuer
net_chg.sort_values(ascending=False).head(n=10)

Issuer
Blackrock Financial Management       5.386207e+09
Vanguard                             5.199436e+09
Invesco                              1.568226e+09
Main Management                      1.019558e+09
State Street                         9.633140e+08
Jpmorgan Chase                       4.984712e+08
Dimensional                          3.654547e+08
Charles Schwab                       3.620408e+08
Allianz Investment Management LLC    2.812078e+08
Goldman Sachs                        2.215761e+08
Name: VALUE, dtype: float64

In [394]:
#top fund outflow by issuer
net_chg.sort_values(ascending=True).head(n=10)

Issuer
Vaneck                       -135858266.0
Tortoise                       -4796531.0
Advisors Asset Management      -1494906.0
Virtus Investment Partners      -891437.0
Alpha Architect                 -881766.0
Sprott                          -825005.0
Fcf Advisors                    -778673.0
Nationwide                      -525186.0
Inspire Investing LLC           -475944.0
Us Global Investors             -416482.0
Name: VALUE, dtype: float64

In [404]:
metric  = 'ETF Security Description'
net_chg_market = df_3.loc['2023-06-30'].groupby(metric)['VALUE'].sum() - df_3.loc['2023-03-31'].groupby(metric)['VALUE'].sum() 
net_chg_market.sort_values(ascending=False).head(n=10) #fund flow by market, change 'ETF Security Description' to 'Asset Class' or other parameters, change VALUE to Holding Weight for $ vs % 

ETF Security Description
VANGUARD TOTAL STOCK MARKET ETF         1.306409e+09
VANGUARD INDEX FDS S&P 500 ETF          1.125356e+09
NORTHERN LTS FD TR IV MAIN SECTOR R     8.388741e+08
ISHARES RUSSELL 1000 GR ETF             8.278449e+08
ISHARES IBOXX $ INVESTMENT GRADE COR    7.437847e+08
Name: VALUE, dtype: float64

In [408]:
df_3.loc['2023-06-30'].groupby(metric)['VALUE'].sum().sort_values(ascending=False).head(n=10) #rank by AUM top 10

ETF Security Description
VANGUARD INDEX FDS S&P 500 ETF          10572692488
VANGUARD TOTAL STOCK MARKET ETF          8433023788
ISHARES CORE S&P 500 ETF                 4161900779
ISHARES RUSSELL 1000 GR ETF              3707784062
ISHARES BARCLAYS 7-10 YEAR TREASURY      3606064652
ISHARES TR RUSSELL 1000 ETF              3054379667
ISHARES NATIONAL MUNI BOND ETF           2864316496
VANGUARD FTSE EMERGING MKTS ETF EMER     2858592506
ISHARES U.S. TREASURY BOND ETF           2674890083
SPDR S&P 500 ETF TR UNIT S               2585230048
Name: VALUE, dtype: int64

In [447]:
market = 'Asset Class'
subcat = 'Real Estate'
#top products in each market by aum
cleanETFtableCompare.set_index([
    'Period of Report',
    market, 
    'ETF Security Description',
    'Company Name'
                                ],drop=False).loc['2023-06-30'].loc[subcat]['VALUE'].groupby('ETF Security Description').sum().sort_values(ascending=False).head(n=10)


ETF Security Description
VANGUARD SPECIALIZED FUNDS VANGUARD     264694356
VANGUARD GLOBAL EX- U.S. REAL ESTATE    139972910
AMERICAN CENTY ETF TR AVANTIS REAL E     24957886
ISHARES TR US REAL ESTATE ETF            20804786
ISHARES GLOBAL REIT ETF                  16324635
SELECT SECTOR SPDR TR REAL ESTATE SE     12829200
SCHWAB STRATEGIC U.S. REIT ETF           12694171
ISHARES TR COHEN & STEERS REIT ETF       11915852
ISHARES TR INTL DEVELOPED PPTY ETF       10426425
ISHARES CORE U.S. REIT ETF                6735760
Name: VALUE, dtype: int64

In [455]:
df4 = cleanETFtableCompare.set_index([
    'Period of Report',
    market, 
    'ETF Security Description',
    'Company Name'
                                ],drop=False)
b4_4 = df4.loc['2023-03-31'].loc[subcat]['VALUE'].groupby('ETF Security Description').sum()
af_4 = df4.loc['2023-06-30'].loc[subcat]['VALUE'].groupby('ETF Security Description').sum()
(af_4-b4_4).dropna().sort_values(ascending=False).head(n=10) #rank by fund flow


ETF Security Description
INVESCO ETF TR II KBW PREMIUM YIELD     5285493.0
VANGUARD GLOBAL EX- U.S. REAL ESTATE    5148332.0
SELECT SECTOR SPDR TR REAL ESTATE SE    2924389.0
VANGUARD SPECIALIZED FUNDS VANGUARD     2535030.0
ISHARES TR COHEN & STEERS REIT ETF      2408688.0
ISHARES GLOBAL REIT ETF                 1974101.0
AMERICAN CENTY ETF TR AVANTIS REAL E    1566272.0
SPDR DOW JONES INTERNATIONAL REAL ES     800551.0
ISHARES CORE U.S. REIT ETF               655980.0
ISHARES INTERNATIONAL DEVELOPED REAL     495100.0
Name: VALUE, dtype: float64